# This Notebook demonstrates how to convert geonames coordinates from WGS84 to the Austrian Lambert Conformal Conic projection (EPSG:31287) and save the results

In [3]:
import numpy as np
import pandas as pd

## reading and cleaning data

In [4]:
#https://www.kaggle.com/datasets/geonames/geonames-database?select=geonames.csv
raw = pd.read_csv('assets/geonames.csv', delimiter=',')
raw.head()

C:\Users\flori\AppData\Local\Temp\ipykernel_22076\147038192.py:2: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv('assets/geonames.csv', delimiter=',')


,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,2986043,Pic de Font Blanca,Pic de Font Blanca,Pic de Font Blanca;Pic du Port,42.64991,1.53335,T,PK,AD,NaN,00,NaN,NaN,NaN,0,NaN,2860,Europe/Andorra,2014-11-05
1,2994701,Roc Mélé,Roc Mele,Roc Mele;Roc Meler;Roc Mélé,42.58765,1.74028,T,MT,AD,AD;FR,00,NaN,NaN,NaN,0,NaN,2803,Europe/Andorra,2014-11-05
2,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD,AD;FR,00,NaN,NaN,NaN,0,NaN,2685,Europe/Andorra,2014-11-05
3,3017832,Pic de les Abelletes,Pic de les Abelletes,Pic de la Font-Negre;Pic de la Font-Nègre;Pic ...,42.52535,1.73343,T,PK,AD,FR,A9,66,663.0,66146.0,0,NaN,2411,Europe/Andorra,2014-11-05
4,3017833,Estany de les Abelletes,Estany de les Abelletes,Estany de les Abelletes;Etang de Font-Negre;Ét...,42.52915,1.73362,H,LK,AD,FR,A9,NaN,NaN,NaN,0,NaN,2260,Europe/Andorra,2014-11-05


In [5]:
print(raw.dtypes)
print('-'*50)
print(raw.isnull().sum())

geonameid              int64
name                  object
asciiname             object
alternatenames        object
latitude             float64
longitude            float64
feature class         object
feature code          object
country code          object
cc2                   object
admin1 code           object
admin2 code           object
admin3 code           object
admin4 code           object
population             int64
elevation            float64
dem                    int64
timezone              object
modification date     object
dtype: object
--------------------------------------------------
geonameid                   0
name                        7
asciiname                 122
alternatenames        5567277
latitude                    0
longitude                   0
feature class            4572
feature code            78893
country code            13767
cc2                  10745667
admin1 code             45752
admin2 code           5523969
admin3 code           91

select relevant columns and create new data frame

In [6]:
# select relevant columns
df = raw[['name', 'latitude', 'longitude', 'country code']]

# filter only Austria
df = df[df['country code'] == 'AT']

# show result
print(df.head())

                name  latitude  longitude country code
244036   Sandgatterl     47.75   14.56667           AT
244037    Viehtalalm     47.75   14.56667           AT
244038  Adlmoarstein     47.75   14.55000           AT
244039  Waldbaueralm     47.75   14.56667           AT
244040      Federeck     47.75   14.56667           AT


remove duplicates and NaNs

In [7]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['name'], keep='first',inplace=True)
df.isnull().sum()

print(df.head())

                name  latitude  longitude country code
244036   Sandgatterl     47.75   14.56667           AT
244037    Viehtalalm     47.75   14.56667           AT
244038  Adlmoarstein     47.75   14.55000           AT
244039  Waldbaueralm     47.75   14.56667           AT
244040      Federeck     47.75   14.56667           AT


## Conversion

This is the lambert conversion function

### Details:
1. Constants for the Austrian Lamber system:
- E0, N0: False easting and northing (400.000 m)
- FSP, SSP: First and second standard parallels (49°N, 46°N)
- LoO: Latitude of Origin (47,5°N)
- CM: Central meridian (13,3333°E)
- R: Semi-major axis of Bessel 1841 ellipsoid (6377397,155 m)

2. Convert degrees to radians:

    Needed for trigonomic calculations.

3. Compute cone constant n:
    
    Determines the convergance of meridians for the conic projections.

        n = np.log(np.cos(phi1) / np.cos(phi2)) / np.log(np.tan(np.add(np.pi/4, phi2/2)) / np.tan(np.add(np.pi/4, phi1/2)))

4. Compute scale factor F:

    Scales the projection to match the standard parallels.

5. Compute reference radius p0:
    
    Distance from the origin latitude to the projection plane.

6. Compute radius p for each input latitude:

    Distance from each point to the projection center.

7. Compute angular difference theta:

    Difference in longitude from the central meridian, scaled by n.

8. Compute projected coordinates (x, y):

    Using the formulas for a Conformal Conic projection:

        x = np.add(E0, np.multiply(p, np.sin(theta))) # Easting
        y = np.add(N0, np.subtract(p0, np.multiply(p, np.cos(theta))))  # Northing


### Notes
- Works with vectorized NumPy arrays, so it can process millions of points efficiently.
- Returns coordinates in meters relative to the false origin (E0, N0).
- Only valid for the Austrian Lamber system (EPSG:31287)

In [8]:
def wgs84_to_lambert (lon: np.array , lat : np.array) -> tuple :
    """ Convert WGS 84 coordinates to an Austrian Lambert Conic Conformal
    Projection ( lcc)
    EPSG code : 31287 , see https :// epsg .io /31287

    Parameters :
        lon (np. array ): Vector containing wgs 84 longitudes
        lat (np. array ): Vector containing wgs 84 latitudes

    Returns :
        x, y (np. array ): Vectors containing lcc coordinates
    """

    # Constants
    E0 = 400000 # in meters
    N0 = 400000 # in meters
    FSP = 49 # fsp = First Standard Parallel ('PHI1') 49°N
    SSP = 46 # ssp = second standard parallel ('PHI2') 46°N
    LoO = 47.5 # loo = latitude of origin ('PHI0') 47.5°N
    CM = 13.3333 # cm = central meridian ('lambda0') 13°20'E (13.3333333333°E)
    
    # Convertions
    phi1 = np.radians(FSP)
    phi2 = np.radians(SSP)
    phi0 = np.radians(LoO)
    phi = np.radians(lat)
    lm0 = np.radians(CM) # lambda0
    lm = np.radians(lon) # lambda
    R = 6377397.155 # Semi-major axis (Bessel 1841)
    
    # Step 1: Compute the Cone Constant 'n'
    n = np.log(np.cos(phi1) / np.cos(phi2)) / np.log(np.tan(np.add(np.pi/4, phi2/2)) / np.tan(np.add(np.pi/4, phi1/2)))
    #print(f'n = {n}')
    
    # Step 2: Compute the Scale Factor 'F'
    F = np.multiply(np.cos(phi1), np.power(np.tan(np.add(np.pi/4, phi1/2)), n)) / n
    #print(f'F = {F}')
    
    # Step 3: Compute the Reference Radius 'p0'
    p0 = np.multiply(R, F) / np.power(np.tan(np.add(np.pi/4, phi0/2)), n)
    #print(f'p0 = {p0}')
    
    # Step 4: Compute the Radius for the given Latitude
    p = np.multiply(R, F) / np.power(np.tan(np.add(np.pi/4, phi/2)), n)
    #print(f'p = {p}')
    
    # Step 5: Compute the Angular Difference from the Central Meridian (theta)
    theta = np.multiply(n, np.subtract(lm, lm0))
    #print(f'theta ={theta}') 
    
    # Step 6: Compute the Final Projected Coordinates: E, N; E0, N0 = false Easting and false Northing
    x = np.add(E0, np.multiply(p, np.sin(theta))) # Easting
    y = np.add(N0, np.subtract(p0, np.multiply(p, np.cos(theta))))  # Northing
    #print(f'Easting (E) = {x}')
    #print(f'Northing (N) = {y}')
    
    
    return x, y

## Save as csv

In [9]:
df['lambert_x'], df['lambert_y'] = wgs84_to_lambert(df['longitude'].to_numpy(), df['latitude'].to_numpy())

# save as CSV
df[['name', 'lambert_x', 'lambert_y']].to_csv('lambert_coord.csv', index=False)


print(df[df['name']== 'Wien'])

        name  latitude  longitude country code      lambert_x      lambert_y
253103  Wien   48.2082   16.37169           AT  625262.682938  483206.953585
